In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
!apt install enchant
!pip install pyenchant
!pip3 install pyenchant==1.6.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
enchant is already the newest version (1.6.0-11.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import nltk
import enchant
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
from gensim import models
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import re
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
train=pd.read_csv("train.csv",encoding='latin-1')
datos =pd.read_csv("test.csv",encoding='latin-1')
datos.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [ ]:
def tokenizarTexto(x):
  x=x.casefold()
  d = enchant.Dict("en")
  x=re.sub(r'http\S*', '',x)
  x=re.sub(r'[^a-z\s]', '',x)
  lemma=WordNetLemmatizer()
  x=lemma.lemmatize(x)
  tokens=nltk.word_tokenize(x)
  #tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
  #tokens = tknzr.tokenize(x)
  l=stopwords.words('english')+list(punctuation)
  tokens = [word for word in tokens if word not in l]
  tokens= [word for word in tokens if d.check(word)]
  tokens= [word for word in tokens if wordnet.synsets(word)]
  return tokens

In [ ]:
import re
def countVect (train,datos):
  lenTrain=len(train)
  lenDatos=len(datos)
  df=pd.concat([train,datos])
  vectorizer=CountVectorizer(tokenizer=tokenizarTexto)
  x = vectorizer.fit_transform(df['text'])
  count_vect_df = pd.DataFrame(x.todense(), columns=vectorizer.get_feature_names())
  dftrain = count_vect_df.iloc[0:lenTrain]
  dfdatos = count_vect_df.iloc[lenTrain:]
  return (dftrain,dfdatos)

(dftrain,dfdatos) = countVect(train,datos)
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Columns: 9901 entries, ab to zoom
dtypes: int64(9901)
memory usage: 575.1 MB


In [ ]:
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import FeatureHasher
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.metrics import f1_score
from matplotlib import pyplot
from sklearn.preprocessing import LabelEncoder
clf = svm.SVC()
#clf=MLPRegressor(random_state=1, max_iter=500)
#Construimos los datos de entrenamiento y de test
X = dftrain
y = train["target"]

"""sel = SelectFromModel(RandomForestClassifier(n_estimators = 200))
sel.fit(X, y)
selCol= X.columns[(sel.get_support())]"""

#X=entrenar[selCol]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
# estandarizamos las features
scaler = StandardScaler()

# Don't cheat - fit only on training data
scaler.fit(X_train)  # doctest: +SKIP
X_train = scaler.transform(X_train)  # doctest: +SKIP
# apply same transformation to test data
X_test = scaler.transform(X_test)  # doctest: +SKI

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
#y_pred = (y_pred>0.5)*1
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 77.08%


In [ ]:
scaler = StandardScaler()

# Don't cheat - fit only on training data
scaler.fit(X)  # doctest: +SKIP
x_train= scaler.transform(X)  # doctest: +SKIP
# apply same transformation to test data
clf.fit(x_train, y)
#test=test[selCol]
test=scaler.fit_transform(dfdatos)
preds = clf.predict(test)
predictions = pd.DataFrame()
predictions['id']=datos['id']
predictions['target']=preds
predictions.head()
predictions.to_csv('resultados.csv',index=False)

In [ ]:
#!kaggle competitions submit -c nlp-getting-started -f resultados.csv -m "Nuevo intento"

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
